<a href="https://colab.research.google.com/github/Tsurielvizel/heb_reminder/blob/main/heb_reminder_ics_maker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install convertdate
!pip install ics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 46.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 2.3 MB/s eta 0:00:00
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732003 sha256=55aee53c34afa7ad22a42ecf288f7d10c5d73322bbb7b1491933e90d6b06aafe
  Stored in directory: /root/.cache/pip/wheels/d6/67/78/aa2e8d108639dd23a5e9e72a4fc88bb44f5541894382712f48
Successfully built pymeeus
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
import re
from convertdate import hebrew, gregorian
import datetime
import ics




In [ ]:
'''
this is a tishrei to a elul in Nisan startiing years
5784, 7, 1 tishrei
5784, 8, 1
5784, 9, 1
5784, 10, 1
5784, 11, 1
5784, 12, 1 adar a (or regular adar in regular year)
5784, 13, 1 adar b (if exist)
5784, 1, 1 Nisan
5784, 2, 1
5784, 3, 1
5784, 4, 1
5784, 5, 1
5784, 6, 1 elul

in a regular year
5785, 7, 1 tishrei
5785, 8, 1
5785, 9, 1
5785, 10, 1
5785, 11, 1
5785, 12, 1 adar
5785, 1, 1 Nisan
5785, 2, 1
5785, 3, 1
5785, 4, 1
5785, 5, 1
5785, 6, 1 elul
'''
def to_ecclestical(year, month, day): # to check if working good because i assume the leap year as start from tishrei year
    """Convert a date in the civil calendar (year starts in Tishrei) to
    the ecclestical calendar (year starts in Nisan)."""
    if(hebrew.year_months(year)==12):
        if month <= 6:
            month = month+6
        else:
            month = month-6
    else:
        if month <= 7:
            month = month+6
        else:
            month = month-6
    return year, month, day

In [ ]:

# רשימת חודשים עבריים
'''
HEBREW_MONTHS = {
    "תשרי": "Tishrei", "חשוון": "Cheshvan", "מרחשוון": "Cheshvan",
    "כסלו": "Kislev", "טבת": "Tevet", "שבט": "Shevat",
    "אדר": "Adar", "אדר א": "Adar I", "אדר ב": "Adar II",
    "ניסן": "Nisan", "אייר": "Iyar", "סיוון": "Sivan",
    "תמוז": "Tamuz", "אב": "Av", "אלול": "Elul"
}'''

HEBREW_MONTHS = {
    "תשרי": 7,"תישרי": 7,
    "חשוון": 8,"חשון": 8, "מרחשוון": 8,"מרחשון": 8,
    "כסלו": 9, "כיסלו": 9,
      "טבת": 10, "שבט": 11,
    "אדר": 12, "אדר א": 12, "אדר ב": 13,
    "ניסן": 1, "אייר": 2, "סיוון": 3,"סיון": 3,
    "תמוז": 4, "אב": 5, "אלול": 6
}



hebrew_days = {"א": 1, "ב": 2, "ג": 3, "ד": 4, "ה": 5, "ו": 6, "ז": 7, "ח": 8, "ט": 9,
                "י": 10, "יא": 11, "יב": 12, "יג": 13, "יד": 14, "טו": 15, "טז": 16,
                "יז": 17, "יח": 18, "יט": 19, "כ": 20, "כא": 21, "כב": 22, "כג": 23,
                "כד": 24, "כה": 25, "כו": 26, "כז": 27, "כח": 28, "כט": 29, "ל": 30}
# גימטריא

HEBREW_GEMATRIA = {
    'א': 1, 'ב': 2, 'ג': 3, 'ד': 4, 'ה': 5, 'ו': 6, 'ז': 7, 'ח': 8, 'ט': 9,
    'י': 10, 'כ': 20, 'ל': 30, 'מ': 40, 'נ': 50, 'ס': 60, 'ע': 70, 'פ': 80,
    'צ': 90, 'ק': 100, 'ר': 200, 'ש': 300, 'ת': 400
}

# פונקציה לניקוי התאריך העברי (עם גרשיים או עם מילים)
def clean_hebrew_date(hebrew_date):
    """
    מנקה את התאריך העברי ומחזיר אותו בפורמט אחיד (כגון '18 Sivan 5784').

    Args:
        hebrew_date (str): תאריך עברי לדוגמה "י"ח סיוון תשפ"ד" או "יח סיוון תשפ"ד" או "ג אדר ב תשפ\"ה".

    Returns:
        str: תאריך עברי נקי ומנורמל בפורמט '18 Month Year'.
        כלומר ה סיוון תשפה מוחזר כ
          '5 Sivan 5785'

    Raises:
        ValueError: אם הפורמט שגוי או אם החודש אינו מוכר.
    """

    # פיצול הקלט לשנה חודש ויום

    # פיצול התאריך לשלושה או ארבעה חלקים
    parts = hebrew_date.split()
    if len(parts) != 3 and len(parts) != 4:
        raise ValueError(f"תאריך לא תקני: {hebrew_date}")

    day, month_name, year = parts[:3]  # אם יש 3 חלקים, הפונקציה תקבל את שלושת החלקים הראשונים

    # אם יש 4 חלקים, , תיקון השנה והוספת חלק נוסף לחודש
    if len(parts) == 4:
        month_name = f"{parts[1]} {parts[2]}"  # אם מדובר באדר א' או אדר ב'
        year = parts[3]


# טיפול בכל חלק של הקלט בנפרד

    # טיפול ביום
    day = re.sub(r"['״]", "", day)  # הסרת גרשיים וגרשיים כפולים
    if day.isdigit():  # אם היום כבר מספר
        day = int(day)
    else:  # המרת מילים (כמו יח) למספרים
        if day in hebrew_days:
            day = hebrew_days[day]
        else:
            raise ValueError(f"היום לא תקני: {day}")

    # טיפול בחודש
    month_name = re.sub(r"['״]", "", month_name)  # הסרת גרשיים
    if month_name not in HEBREW_MONTHS:
        raise ValueError(f"חודש לא תקני: {month_name}")
    month = HEBREW_MONTHS[month_name]

    #טיפול בשנה
    # ניקוי גרשיים מהשנה
    year = re.sub(r"['״]", "", year)
    if not (year.startswith("תש") or year.startswith("תת")):
        raise ValueError(f"שנה לא תקנית: {year}")
    else:
        # חישוב הגימטריה של השנה
    # חישוב הגימטריה עבור שאר האותיות
        year_value = 5000
        for char in year:
            if char in HEBREW_GEMATRIA:
                year_value += HEBREW_GEMATRIA[char]



    # החזרת התאריך בפורמט שנה שמתחיל בניסן (eccelntar year)
    return year_value, month, day



In [ ]:
# פונקציה להמיר תאריך עברי לתאריכים לועזיים
def hebrew_to_gregorian_dates(start_hebrew_date, years=1):
    """
    מקבלת תאריך עברי ומחשבת את כל התאריכים הלועזיים עבור מספר שנים.

    Args:
        start_hebrew_date (str):
          תאריך עברי בפורמט
          יח סיוון תשפד

        years (int): מספר השנים להמיר.

    Returns:
        list: רשימה של תאריכים לועזיים בפורמט 'YYYY-MM-DD'.
    """
    year, month, day  = clean_hebrew_date(start_hebrew_date)

    # המרת התאריך העברי לגלוגריאני
    #  ויצירת מערך של התאריכים הלועזיים
    gregorian_dates = []
    for year_offset in range(years + 1):
        # המרת כל שנה
            gregorian_date = hebrew.to_gregorian(year + year_offset, month, day)
            gregorian_dates.append(str(gregorian_date))
    print(f'gregorian_dates is {gregorian_dates}')
    return gregorian_dates



In [ ]:
import ics
import datetime

def tuple_date_to_string(date_tuple):
    # המרת המיתר לפורמט טופל (למשל "(2025, 6, 14)" ל-(2025, 6, 14))
    date_tuple = eval(date_tuple)  # המרה מיתרים לטופל

    # הפיכת tuple לפורמט YYYY-MM-DD
    year = int(date_tuple[0])  # המרת שנה למספר שלם
    month = int(date_tuple[1])  # המרת חודש למספר שלם
    day = int(date_tuple[2])  # המרת יום למספר שלם

    data_str = f"{year}-{month:02d}-{day:02d}"  # הפיכת tuple לפורמט YYYY-MM-DD
    return data_str

def date_format_create(gregorian_dates, time):
    """
    Convert a list of Gregorian date tuples to a formatted list of datetime strings.

    Args:
        gregorian_dates: A list of strings, each containing a tuple like '(YYYY, M, D)'.
        time: A string representing the time in "HH:MM" format.

    Returns:
        A list of strings formatted as "YYYY-MM-DD HH:MM".
    """
    formatted_dates = []

    for date_str in gregorian_dates:
        date_str = tuple_date_to_string(date_str)  # המרת כל מיתר לטופל ואז לפורמט
        formatted_dates.append(f"{date_str} {time}")

    return formatted_dates





# פונקציה ליצור קובץ ICS עם התאריכים הלועזיים
def create_ics_file(dates, summary='תזכורת', time='10:00'):
    """
    Creates an ICS file for multiple events.

    Args:
        dates: A list of dates, either as tuples (YYYY, M, D) or 'YYYY-MM-DD' strings.
        summary: The summary or title of the event.
        time: The time of the event in 'HH:MM' format.

    Returns:
        An ICS file as a string.
    """
    c = ics.Calendar()

    # Add events for each date
    for i in range(len(dates)):
        event = ics.Event()
        event.name = summary
       # date_time = datetime.date(int(date[0]),int(date[1]),int(date[2]))
        # המרת tuple לפורמט "YYYY-MM-DD" אם צריך
        #date_str = tuple_dato_to_string(date)
        # יצירת datetime עם הפורמט המתאים
     #   event_datetime_str = f"{date_str} {time}"
        data = date_format_create(dates, time)
        event.begin = datetime.datetime.strptime(data[i], "%Y-%m-%d %H:%M")

        # קביעת זמן סיום לאירוע (שעה אחת אחרי ההתחלה)
        event.end = event.begin + datetime.timedelta(hours=1)
        c.events.add(event)

    return c.serialize()



In [ ]:
# פונקציה לשאול את המשתמש ולהפיק תזכורת ליומן גוגל
def create_google_calendar_reminder():

    """
    מבקשת תאריך עברי, מספר שנים ושעה, ומחזירה קובץ ICS עם התזכורות המתאימות ליומן גוגל.
    """

    # קבלת פרטי התזכורת מהמשתמש
    hebrew_date = input("הכנס תאריך עברי (לדוג' יח סיוון תשפ\"ה): ")
    years = int(input("לכמה שנים אתה רוצה את התזכורת? "))
    time = input("באיזו שעה אתה רוצה את התזכורת? (לדוג' 10:00): ")


    # המרת התאריך לעברית לגלוגריאני
    gregorian_dates = hebrew_to_gregorian_dates(hebrew_date, years)
    # יצירת קובץ ICS עם התאריכים הלועזיים
    ics_data = create_ics_file(gregorian_dates,summary="תזכורת יומית", time=time)

    # שמירת הקובץ
    with open("reminder.ics", "w") as f:
        f.write(ics_data)

    print("הקובץ נוצר בהצלחה בשם 'reminder.ics'. אתה יכול לייבא אותו ליומן גוגל שלך.")

In [ ]:
create_google_calendar_reminder()

הכנס תאריך עברי (לדוג' יח סיוון תשפ"ה): יט כסלו תשפו
לכמה שנים אתה רוצה את התזכורת? 2
באיזו שעה אתה רוצה את התזכורת? (לדוג' 10:00): 10:00
gregorian_dates is ['(2025, 12, 9)', '(2026, 11, 29)', '(2027, 12, 19)']
הקובץ נוצר בהצלחה בשם 'reminder.ics'. אתה יכול לייבא אותו ליומן גוגל שלך.


In [ ]:
from google.colab import files
files.download("reminder.ics")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib icalendar


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0


In [ ]:
import os
from google.colab import auth
from googleapiclient.discovery import build
from oauth2client import file, client
from icalendar import Calendar
import logging

# הגדרת רמת הלוגינג
logging.basicConfig(level=logging.INFO)

# פונקציה לאימות
def authenticate_google_calendar():
    SCOPES = ['https://www.googleapis.com/auth/calendar.events']

    if 'COLAB_GPU' in os.environ:  # אם זה הריצה ב-Colab
        auth.authenticate_user()
        creds = None
    else:  # אם זה במחשב המקומי
        store = file.Storage('token.json')
        creds = store.get()
        if not creds or creds.invalid:
            flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
            creds = client.tools.run_flow(flow, store)

    service = build('calendar', 'v3', credentials=creds)
    return service

# פונקציה להוספת אירוע ליומן Google
def create_event(service, event):
    try:
        event = service.events().insert(calendarId='primary', body=event).execute()
        logging.info(f"Event created: {event.get('htmlLink')}")
    except Exception as e:
        logging.error(f"Error creating event: {e}")

# פונקציה לקריאת קובץ ICS והוספת האירועים ליומן Google
def import_ics_to_google_calendar(service, ics_file):
    with open(ics_file, 'r') as f:
        try:
            calendar = Calendar.from_ical(f.read())
            for component in calendar.walk():
                if component.name == "VEVENT":
                    # בניית האירוע עבור Google Calendar
                    event = {
                        'summary': str(component.get('summary')),
                        # הוסף שדות נוספים לפי הצורך
                        'start': {
                            'dateTime': component.get('dtstart').dt.isoformat(),
                            'timeZone': 'Asia/Jerusalem',
                        },
                        'end': {
                            'dateTime': component.get('dtend').dt.isoformat(),
                            'timeZone': 'Asia/Jerusalem',
                        },
                    }
                    create_event(service, event)
        except Exception as e:
            logging.error(f"Error parsing ICS file: {e}")

# שלב 1: התחברות ל-API של Google Calendar
service = authenticate_google_calendar()

# שלב 2: קריאת קובץ ICS והוספת האירועים
ics_file_path = 'reminder.ics'  # אם בקולאב, תוכל להעלות את הקובץ כאן או במחשב המקומי
import_ics_to_google_calendar(service, ics_file_path)


ERROR:root:Error creating event: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">
ERROR:root:Error creating event: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">
ERROR:root:Error creating event: <HttpError 403 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


In [ ]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib